In [1]:
print("hare krishna")

hare krishna


### Imports

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from pprint import pprint


#### API Keys

In [9]:
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
groq_api_key=os.getenv('GROQ_API_KEY')

TypeError: str expected, not NoneType

#### Embeedings

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="lala",
    openai_api_version="2024-03-01-preview",
)

In [ ]:
!pip install -q langchain-chroma

### Create Vector Data base and store

In [ ]:
from langchain_core.documents import Document
sexual_offences_docs=[
    Document(
        page_content="""
        ###IPC section 375. Rape ###
        —A man is said to commit “rape” if he— 
        (a) penetrates his penis, to any extent, into the vagina, mouth, urethra or anus of a woman or makes her to do so with him or any other person; or 
        (b) inserts, to any extent, any object or a part of the body, not being the penis, into the vagina, the urethra or anus of a woman or makes her to do so with him or any other person; or
        (c) manipulates any part of the body of a woman so as to cause penetration into the vagina, urethra, anus or any part of body of such woman or makes her to do so with him or any other person; or
        (d) applies his mouth to the vagina, anus, urethra of a woman or makes her to do so with him or any other person,
        ###under the circumstances falling under any of the following seven descriptions###— 
        First.—Against her will. 
        Secondly.—Without her consent. 
        Thirdly.—With her consent, when her consent has been obtained by putting her or any person in whom she is interested, in fear of death or of hurt. 
        Fourthly.—With her consent, when the man knows that he is not her husband and that her consent is given because she believes that he is another man to whom she is or believes herself to be lawfully married. 
        Fifthly.—With her consent when, at the time of giving such consent, by reason of unsoundness of mind or intoxication or the administration by him personally or through another of any stupefying or unwholesome substance, she is unable to understand the nature and consequences of that to which she gives consent.

        Sixthly.—With or without her consent, when she is under eighteen years of age. 
        Seventhly.—When she is unable to communicate consent.
        """,
        metadata={
            "law":"IPC Section 375 - Rape",
            "content":"law definition and Conditons of rape IPC Section 375",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###Key notes for IPC section 375 ###
        1) Rape.—Offence of rape is against entire society. It is extremely reprehensible and hated crime which defiles and degrades victim physically as well as mentally and shakes very core of life and dignity, Anurag Soni v. State of Chhattisgarh, (2019) 13 SCC 1.
        2) Intention/Consent/Mens Rea.—“Consent” is stated to be an act of reason coupled with deliberation. It denotes an active will in the mind of a person to permit the doing of an act complained of. Inference as to consent can be drawn on the basis of evidence or probabilities of the case, with regard to relevant circumstances, Kaini Rajan v. State of Kerala, (2013) 9 SCC 113 : (2013) 3 SCC (Cri) 858. 
        3) Underage victim.—Where age of victim was about 14 years at the time of offence, question of consent was of no consequence, State of H.P. v. Shree Kant Shekari, (2004) 8 SCC 153. 
        4) Consensual sexual relationship.—Court must ascertain whether both parties were ad idem on essential features, Vinod Kumar v. State of Kerala, (2014) 5 SCC 678 : (2014) 2 SCC (Cri) 663. 
        5) Consent of prosecutrix to physical relationship.—Consent given under misconception of fact is no consent in eyes of law but, misconception of fact, has to be in proximity of time to occurrence, Maheshwar Tigga v. State of Jharkhand, (2020) 10 SCC 108. 
        """,
        metadata={
            "law":"IPC section 375 - Rape",
            "content":"Key legal interpretations of IPC section 375 - Rape and case notes",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###Explanation for IPC section 375 ###
        Explanation 1.—For the purposes of this section, “vagina” shall also include labia majora. 
        Explanation 2.—Consent means an unequivocal voluntary agreement when the woman by words, gestures or any form of verbal or nonverbal communication, communicates willingness to participate in the specific sexual act: Provided that a woman who does not physically resist to the act of penetration shall not by the reason only of that fact, be regarded as consenting to the sexual activity.
        """,
        metadata={
            "law":"IPC section 375 - Rape",
            "content":"explanation related to IPC 375 and explanations related to consent",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###Exceptions of IPC section 375 ### 
        Exception 1.—A medical procedure or intervention shall not constitute rape.
        Exception 2.—Sexual intercourse or sexual acts by a man with his own wife, the wife not being under fifteen years of age, is not rape
        """,
         metadata={
            "law":"IPC section 375 - Rape",
            "content":"Exceptions to IPC 375 - Rape",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###key pointsof IPC section 375 ### 
        Sex with minor wife.—Sexual intercourse with minor (below 18 years) wife is rape. 
        Exception 2 to Section 375 IPC is arbitrary, capricious, whimsical and violative of Article 14, 15 and 21 of the Constitution of India. However, the verdict will apply prospectively, Independent Thought v. Union of India, (2017) 10 SCC 800. Section 198(6) of the CrPC will apply to cases of rape of “wives” below 18 years, and cognizance can be taken only in accordance with the provisions of Section 198(6) of the Code, Independent Thought v. Union of India, (2017) 10 SCC 800. 
        """,
         metadata={
            "law":"IPC section 375 - Rape",
            "content":"IPC section 375 - Rape key-point -(Sex with minor wife)",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###keypoint of IPC section 375 ### 
        ► Meaning of “consent” with respect to Section 375.—“Consent” with respect to Section 375 involves an active understanding of circumstances, actions and consequences of proposed act. Individual who makes a reasoned choice to act after evaluating various alternative actions (or inaction) as well as various possible consequences flowing from such action or inaction, consents to such action, Pramod Suryabhan Pawar v. State Of Maharashtra, (2019) 9 SCC 608. 
        ► Consent under misconception of fact.—While Section 90 of Penal Code does not define term “consent”, “consent” based on a “misconception of fact” is not consent in the eye of law. Thus, in case of woman engaging in sexual relations on false promise to marriage, her “consent” is based on “misconception of fact”, and such sexual act(s) will amount to rape, Pramod Suryabhan Pawar v. State Of Maharashtra, (2019) 9 SCC 608, See also Anurag Soni v. State of Chhattisgarh, (2019) 13 SCC 1.
        """,
         metadata={
            "law":"IPC section 375 - Rape",
            "content":"IPC section 375 - Rape keypoint - (meaning of consent) and (consent under misconception of fact) ",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###State Amendments if IPC 375 ###
        (Made prior to amendment by Central Act 43 of 1983) Manipur.—In its application to the Union Territory of Manipur, in Section 375— (a) in clause Fifthly (now Sixthly), for the word “sixteen*” substitute the word “fourteen”; and (b) in the Exception, for the word “fifteen” substitute the word “thirteen”. [Vide Act 30 of 1950, S. 3(2) (16-4-1950)] * Amended to “eighteen” by Criminal Law (Amendment) Act, 2013 (13 of 2013).

        """,
        metadata={
            "law":"IPC section 375 - Rape",
            "content":"State amendments to IPC 375 in Manipur, including historical changes in age of consent",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###IPC Section 376 – Punishment for rape ###
        [(1) Whoever, except in the cases provided for in sub-section (2), commits rape, shall be punished with rigorous imprisonment of either description for a term which 384[shall not be less than ten years, but which may extend to imprisonment for life, and shall also be liable to fine]
        [(2) Whoever,— (a) being a police officer, commits rape— (i) within the limits of the police station to which such police officer is appointed; or (ii) in the premises of any station house; or (iii) on a woman in such police officer's custody or in the custody of a police officer subordinate to such police officer; or (b) being a public servant, commits rape on a woman in such public servant's custody or in the custody of a public servant subordinate to such public servant; or (c) being a member of the armed forces deployed in an area by the Central or a State Government commits rape in such area; or (d) being on the management or on the staff of a jail, remand home or other place of custody established by or under any law for the time being in force or of a women's or children's institution, commits rape on any inmate of such jail, remand home, place or institution; or (e) being on the management or on the staff of a hospital, commits rape on a woman in that hospital; or (f) being a relative, guardian or teacher of, or a person in a position of trust or authority towards the woman, commits rape on such woman; or (g) commits rape during communal or sectarian violence; or (h) commits rape on a woman knowing her to be pregnant; or
        (j) commits rape, on a woman incapable of giving consent; or (k) being in a position of control or dominance over a woman, commits rape on such woman; or (l) commits rape on a woman suffering from mental or physical disability; or (m) while committing rape causes grievous bodily harm or maims or disfigures or endangers the life of a woman; or (n) commits rape repeatedly on the same woman]
        [(3) Whoever, commits rape on a woman under sixteen years of age shall be punished with rigorous imprisonment for a term which shall not be less than twenty years, but which may extend to imprisonment for life, which shall mean imprisonment for the remainder of that person's natural life, and shall also be liable to fine: Provided that such fine shall be just and reasonable to meet the medical expenses and rehabilitation of the victim: Provided further that any fine imposed under this sub-section shall be paid to the victim.]
        - shall be punished with rigorous imprisonment for a term which shall not be less than ten years, but which may extend to imprisonment for life, which shall mean imprisonment for the remainder of that person's natural life, and shall also be liable to fine.

        """,
        metadata={
            "law":"IPC section 376 - Punishment for rape",
            "content":"Punishments and aggravated circumstances under IPC section 376, including rape of minors and specific categories of offenders",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###Explanation ###
        Explanation.—For the purposes of this sub-section,— (a) “armed forces” means the naval, military and air forces and includes any member of the Armed Forces constituted under any law for the time being in force, including the paramilitary forces and any auxiliary forces that are under the control of the Central Government or the State Government; 
        (b) “hospital” means the precincts of the hospital and includes the precincts of any institution for the reception and treatment of persons during convalescence or of persons requiring medical attention or rehabilitation; (c) “police officer” shall have the same meaning as assigned to the expression “police” under the Police Act, 1861 (5 of 1861); (d) “women's or children's institution” means an institution, whether called an orphanage or a home for neglected women or children or a widow's home or an institution called by any other name, which is established and maintained for the reception and care of women or children.]

        """,
        metadata={
            "law":"IPC section 376 - Punishment for rape",
            "content":"Definitions and explanations related to armed forces, hospitals, police officers, and women's or children's institutions under IPC section 376",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###Key Points for IPC section 376 ###
        ► Rape of young girl children.—When statutory provisions regarding rape of minor girls and punishment therefor exists under Section 376(2)(i) IPC, Court can neither enhance punishment so provided nor can it create a new offence and prescribed punishment in that regard. This is a legislative function and Court can only make suggestions to legislature, Supreme Court Women Lawyers Assn. (SCWLA) v. Union of India, (2016) 3 SCC 680.
        ► Rape or gang rape — Determination.—A case is not of gang rape, when the evidence does not indicate that the other accused committed rape on the prosecutrix, nor that they shared the common intention to rape nor aided the commission of rape by the person who did commit rape, Manoj Mishra v. State of U.P., (2021) 10 SCC 763.
        ► Statement of prosecutrix — Corroboration when required.—Testimony of the victim in rape cases is vital and unless there are compelling reasons which necessitate looking for corroboration of her statement, the courts should find no difficulty to act on the testimony of a victim of sexual assault alone to convict an accused where her testimony inspires confidence and is found to be reliable, Sham Singh v. State of Haryana, (2018) 18 SCC 34.
        ► DNA test.—Traditional “autosomal-STR” DNA analysis vis-à-vis “Y-STR” method of DNA analysis, distinguished. In cases of sexual assault DNA of victim and the perpetrator are often mixed, and, “Y-STR” method provides a unique way of isolating only the male DNA, hence, “Y-STR” method of DNA analysis preferably be followed, Ravi v. State Of Maharashtra, (2019) 9 SCC 622.
        ► Promiscuity/Habituation to sexual intercourse of prosecutrix.— Promiscuity/habituation to sexual intercourse of prosecutrix cannot be a ground to justify rape, State (NCT of Delhi) v. Pankaj Chaudhary, (2019) 11 SCC 575.
        """,
        metadata={
            "law":"IPC section 376 - Punishment for rape",
            "content":"Legal interpretations on rape of minors, gang rape determination, prosecutrix testimony, DNA analysis methods, and misconceptions about promiscuity/habituation",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###IPC section 376 A ###
        [IPC Section - 376-A. Punishment for causing death or resulting in persistent vegetative state of victim.—Whoever, commits an offence punishable under sub-section (1) or sub-section (2) of Section 376 and in the course of such commission inflicts an injury which causes the death of the woman or causes the woman to be in a persistent vegetative state, shall be punished with rigorous imprisonment for a term which shall not be less than twenty years, but which may extend to imprisonment for life, which shall mean imprisonment for the remainder of that person's natural life, or with death.]

        """,
        metadata={
            "law":"IPC section 376 A - Punishment for causing death or resulting in persistent vegetative state of victim",
            "content":"Definition and legal provisions under IPC section 376-A regarding punishments for causing death or persistent vegetative state of victims",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###State Amendments of IPC section 376-A ###
        Arunachal Pradesh.—In its application to the State of Arunanchal Pradesh, after Section 376-A, the following section shall be inserted, namely— “376-AA. Punishment for rape on a woman up to twelve years of age.—Whoever commits rape on a woman up to twelve years of age shall be punished with death, or rigorous imprisonment for a term which shall not be less than fourteen years but which may extend to imprisonment for life which shall mean imprisonment for the remainder of that person's natural life, and shall also be liable to fine.”. [Vide Arunachal Pradesh Act 3 of 2019, S. 8, dt. 2-4-2019].
        """,
        metadata={
            "law":"IPC section 376 A - Punishment for causing death or resulting in persistent vegetative state of victim",
            "content": "State amendment in Arunachal Pradesh introducing IPC section 376-AA for rape on women up to twelve years of age",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###Key point ###
        ► Applicability of Section 376-A as it stood w.e.f. 03-2-2013 as amended by 2013 Ordinance.—Penal provisions, held, cannot be brought into force with retrospective effect. Thus, held, law as in force as amended by 2013 Ordinance on date of offence would apply. Provisions of 2013 Amendment Act cannot be given retrospective effect as that would violate Article 20(1), Shatrughna Baban Meshram v. State of Maharashtra, (2021) 1 SCC 596.
        """,
        metadata={
            "law":"IPC section 376 A - Punishment for causing death or resulting in persistent vegetative state of victim",
            "content": "Legal interpretation on applicability of IPC section 376-A as per 2013 Ordinance amendments, with insights on retrospective effect and constitutional implications",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###IPC section 376 AB ###
        [376-AB. Punishment for rape on woman under twelve years of age.—Whoever, commits rape on a woman under twelve years of age shall be punished with rigorous imprisonment for a term which shall not be less than twenty years, but which may extend to imprisonment for life, which shall mean imprisonment for the remainder of that person's natural life, and with fine or with death: Provided that such fine shall be just and reasonable to meet the medical expenses and rehabilitation of the victim: Provided further that any fine imposed under this section shall be paid to the victim.]

        """,
        metadata={
            "law":"IPC section 376 AB - Punishment for rape on woman under twelve years of age",
            "content":"Legal definition and provisions under IPC section 376-AB regarding punishments for rape on women under twelve years of age, including sentencing and financial penalties",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###IPC section 376 B  Sexual intercourse by husband upon his wife during separation ###
        [376-B. Sexual intercourse by husband upon his wife during separation.—Whoever has sexual intercourse with his own wife, who is living separately, whether under a decree of separation or otherwise, without her consent, shall be punished with imprisonment of either description for a term which shall not be less than two years but which may extend to seven years, and shall also be liable to fine. Explanation.—In this section, “sexual intercourse” shall mean any of the acts mentioned in clauses (a) to (d) of Section 375.]
        """,
        metadata={
            "law":"IPC section 376 B - Sexual intercourse by husband upon his wife during separation",
            "content":"Legal definition and provisions under IPC section 376-B regarding sexual intercourse by husband with his wife during separation without her consent, including the scope of 'sexual intercourse' as defined in Section 375",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###IPC section 376 C ###
        [376-C. Sexual intercourse by a person in authority.—Whoever, being— 
        (a) in a position of authority or in a fiduciary relationship; or 
        (b) a public servant; or 
        (c) superintendent or manager of a jail, remand home or other place of custody established by or under any law for the time being in force, or a women's or children's institution; or 
        (d) on the management of a hospital or being on the staff of a hospital, abuses such position or fiduciary relationship to induce or seduce any woman either in his custody or under his charge or present in the premises to have sexual intercourse with him, such sexual intercourse not amounting to the offence of rape, shall be punished with rigorous imprisonment of either description for a term which shall not be less than five years, but which may extend to ten years, and shall also be liable to fine.

        """,
        metadata={
            "law":"IPC section 376 C - Sexual intercourse by a person in authority",
            "content":"Legal definition and provisions under IPC section 376-C regarding sexual intercourse by a person in authority, including positions such as public servants, managers of institutions, and hospital staff, with penalties for abusing their position to induce sexual intercourse without constituting rape",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###Explanation for IPC section 376 C ###
        Explanation 1.—In this section, “sexual intercourse” shall mean any of the acts mentioned in clauses (a) to (d) of Section 375.
        Explanation 2.—For the purposes of this section, Explanation 1 to Section 375 shall also be applicable.
        Explanation 3.—“Superintendent”, in relation to a jail, remand home or other place of custody or a women's or children's institution, includes a person holding any other office in such jail, remand home, place or institution by virtue of which such person can exercise any authority or control over its inmates.
        Explanation 4.—The expressions “hospital” and “women's or children's institution” shall respectively have the same meaning as in Explanation to sub-section (2) of Section 376.]
        """,
        metadata={
            "law":"IPC section 376 C - Sexual intercourse by a person in authority",
            "content":"Explanations under IPC section 376-C, detailing the definitions and interpretations related to sexual intercourse by a person in authority, including the meaning of 'sexual intercourse' as per Section 375, applicability of Explanation 1 to Section 375, definitions of 'superintendent' in relation to custodial institutions, and clarification of 'hospital' and 'women's or children's institution'",
            "category":"sexual offences"
        }
    ),
    Document(
        page_content="""
        ###IPC section 376 D ###
        [376-D. Gang rape.—Where a woman is raped by one or more persons constituting a group or acting in furtherance of a common intention, each of those persons shall be deemed to have committed the offence of rape and shall be punished with rigorous imprisonment for a term which shall not be less than twenty years, but which may extend to life which shall mean imprisonment for the remainder of that person's natural life, and with fine: Provided that such fine shall be just and reasonable to meet the medical expenses and rehabilitation of the victim: Provided further that any fine imposed under this section shall be paid to the victim.]
        """,
        metadata={
            "law":"IPC section 376 D - Gang rape",
            "content":"Definition and punishment provisions under IPC section 376-D for gang rape, where multiple persons are involved or act with a common intention in raping a woman. Specifies rigorous imprisonment terms and fines, along with provisions for medical expenses and victim rehabilitation.",
            "category":"sexual offences"
        }
    ),

    Document(
        page_content="""
        ###keypoints for IPC section 376 D ###
        ► Gang rape, Joint liability.—Where a woman is raped by one or more in a group of persons acting in furtherance of their common intention, each of the persons shall be deemed to have committed gang rape and all of them shall be liable to be punished under Section 376(2)(Subsequently amended) IPC, Mukesh v. State (NCT of Delhi), (2017) 6 SCC 1.
        """,
        metadata={
            "law":"IPC section 376 D - Gang rape",
            "content":"Key points on joint liability in gang rape cases under IPC section 376-D. Discusses the legal precedent set by Mukesh v. State (NCT of Delhi), (2017) 6 SCC 1, where each person in a group acting with common intention is liable for gang rape. Emphasizes on the interpretation and application of Section 376(2)(Subsequently amended) IPC.",
            "category":"sexual offences"
        }
    ),

    Document(
        page_content="""
        ###IPC section 376 DA ###
        [376-DA. Punishment for gang rape on woman under sixteen years of age.—Where a woman under sixteen years of age is raped by one or more persons constituting a group or acting in furtherance of a common intention, each of those persons shall be deemed to have committed the offence of rape and shall be punished with imprisonment for life, which shall mean imprisonment for the remainder of that person's natural life, and with fine: Provided that such fine shall be just and reasonable to meet the medical expenses and rehabilitation of the victim: Provided further that any fine imposed under this section shall be paid to the victim.]

        """,
        metadata={
            "law":"IPC section 376 DA - Punishment for gang rape on woman under sixteen years of age",
            "content":"Defines the punishment for gang rape of a woman under sixteen years of age under IPC section 376-DA. Discusses the legal provisions, including the imposition of life imprisonment and fine for each person involved in the group or acting with common intention. Specifies that the fine should be adequate for medical expenses and victim rehabilitation.",
            "category":"sexual offences"
        }
    ),

    Document(
        page_content="""
        Arunachal Pradesh.—In its application to the State of Arunanchal Pradesh, after Section 376-D, the following section shall be inserted, namely— “376-DA. Punishment for gang rape on a woman twelve years of age.—Where a woman up to twelve years of age, is raped by one or more persons constituting a group or acting in furtherance of a common intention, each of those persons shall be deemed to have committed the offence of rape and shall be punished with death, or rigorous imprisonment for a term which shall not be less than twenty years, but which may extend to imprisonment for life which shall mean imprisonment for the remainder of that person's natural life, and shall also be liable to fine: Provided that such fine shall be just and reasonable to meet the medical expenses and rehabilitation of the victim: Provided further that any fine imposed under this section shall be paid to the victim.”. [Vide Arunachal Pradesh Act 3 of 2019, S. 9, dt. 2-4-2019].
        """,
        metadata={
            "law":"IPC section 376 DA - Punishment for gang rape on woman under sixteen years of age",
            "content":"Describes the Arunachal Pradesh amendment to IPC section 376-DA, introducing specific provisions for punishment in cases of gang rape on a woman up to twelve years of age. Specifies the penalties including death or rigorous imprisonment, with provisions for fines to cover medical expenses and victim rehabilitation. Enacted under Arunachal Pradesh Act 3 of 2019, Section 9.",
            "category":"sexual offences"
        }
    ),

    Document(
        page_content="""
        ###IPC section 376 E ###
        [376-E. Punishment for repeat offenders.—Whoever has been previously convicted of an offence punishable under Section 376 or Section 376-A or 395[Section 376-AB or Section 376-D or Section 376- DA or Section 376-DB] and is subsequently convicted of an offence punishable under any of the said sections shall be punished with imprisonment for life which shall mean imprisonment for the remainder of that person's natural life, or with death.]
        """,
        metadata={
            "law":"IPC section 376 E - Punishment for repeat offenders",
            "content":"Defines the punishment for repeat offenders convicted under Section 376, Section 376-A, Section 376-AB, Section 376-D, Section 376-DA, or Section 376-DB of the IPC. Specifies that such offenders shall be punished with imprisonment for life, meaning imprisonment for the remainder of their natural life, or with death penalty.",
            "category":"sexual offences"
        }
    ),

    Document(
        page_content="""
        ###State Amendments for IPC section 376 E ###
        Chhattisgarh.—After Section 376-E, the following shall be inserted, namely— “376-F. Liability of person in-charge of workplace and others to give information about offence.—Whoever, being person in-charge of any work place or any other person present at such place, having knowledge that an offence under Section 376 or Section 376-D, is being committed at such place and being in a position to prevent commission of such offence fails so, to prevent such offence or to give information of the commission of such offence, to any magistrate or police officer, by any mode, with the intention of screening the offender from legal punishment, shall be liable to be punished for abetment of such offence with imprisonment of either description which may extend to three years and fine and no such person shall incur any liability for giving such information. Explanation.—Work place includes any mode of transport owned, hired or otherwise engaged by the person in-charge of the work place for the conveyance of the woman, who was subjected to such offence, to and from her residence to such work place.”. [Vide Chhattisgarh Act 25 of 2015, S. 5, w.e.f. 21-7-2015]
        """,
        metadata={
            "law":"IPC section 376 E - Punishment for repeat offenders",
            "content":"Describes the amendment in Chhattisgarh after Section 376-E, introducing Section 376-F. This new section imposes liability on persons in charge of workplaces or others present at such places, who fail to prevent or report offences under Section 376 or Section 376-D. It specifies abetment punishment with imprisonment up to three years and a fine. Defines workplace to include modes of transport for commuting women to and from their residence.",
            "category":"sexual offences"
        }
    )

]

theft=[
    Document(
        page_content="""
        Whoever, intending to take dishonestly any movable property out of the possession of any person without that person's consent, moves that property in order to such taking, is said to commit theft.
        """,
        metadata={
            "law":"IPC section 378. Theft",
            "content":"law definition of IPC section 378. Theft",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        Explanation 1.—A thing so long as it is attached to the earth, not being movable property, is not the subject of theft; but it becomes capable of being the subject of theft as soon as it is severed from the earth. 
        Explanation 2.—A moving effected by the same act which effects the severance may be a theft. Explanation 3.—A person is said to cause a thing to move by removing an obstacle which prevented it from moving or by separating it from any other thing, as well as by actually moving it.
        Explanation 4.—A person, who by any means causes an animal to move, is said to move that animal, and to move everything which, in consequence of the motion so caused, is moved by that animal.
        Explanation 5.—The consent mentioned in the definition may be express or implied, and may be given either by the person in possession, or by any person having for that purpose authority either express or implied.
        """,
        metadata={
            "law":"IPC section 378. Theft",
            "content":"Explanations for the IPC act 378",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        Illustrations for IPC act 378 -
        (a) A cuts down a tree on Z's ground, with the intention of dishonestly taking the tree out of Z's possession without Z's consent. Here, as soon as A has severed the tree in order to such taking, he has committed theft.
        (b) A puts a bait for dogs in his pocket, and thus induces Z's dog to follow it. Here, if A's intention be dishonestly to take the dog out of Z's possession without Z's consent, A has committed theft as soon as Z's dog has begun to follow A. 
        (c) A meets a bullock carrying a box of treasure. He drives the bullock in a certain direction, in order that he may dishonestly take the treasure. As soon as the bullock begins to move, A has committed theft of the treasure.
        (d) A being Z's servant, and entrusted by Z with the care of Z's plate, dishonestly runs away with the plate, without Z's consent. A has committed theft.
        (e) Z, going on a journey, entrusts his plate to A, the keeper of a warehouse, till Z shall return. A carries the plate to a goldsmith and sells it. Here the plate was not in Z's possession. It could not therefore be taken out of Z's possession, and A has not committed theft, though he may have committed criminal breach of trust. 
        (f) A finds a ring belonging to Z on a table in the house which Z occupies. Here the ring is in Z's possession, and if A dishonestly removes it, A commits theft.
        (g) A finds a ring lying on the high road, not in the possession of any person. A, by taking it, commits no theft, though he may commit criminal misappropriation of property. 
        (h) A sees a ring belonging to Z lying on a table in Z's house. Not venturing to misappropriate the ring immediately for fear of search and detection, A hides the ring in a place where it is highly improbable that it will ever be found by Z, with the intention of taking the ring from the hiding place and selling it when the loss is forgotten. Here A, at the time of first moving the ring, commits theft. 
        (i) A delivers his watch to Z, a jeweller, to be regulated. Z carries it to his shop. A, not owing to the jeweller any debt for which the jeweller might lawfully detain the watch as a security, enters the shop openly, takes his watch by force out of Z's hand, and carries it away. Here A, though he may have committed criminal trespass and assault, has not committed theft, inasmuch as what he did was not done dishonestly.
        (j) If A owes money to Z for repairing the watch, and if Z retains the watch lawfully as a security for the debt, and A takes the watch out of Z's possession, with the intention of depriving Z of the property as a security for his debt, he commits theft, inasmuch as he takes it dishonestly. (k) Again, if A, having pawned his watch to Z, takes it out of Z's possession without Z's consent, not having paid what he borrowed on the watch, he commits theft, though the watch is his own property inasmuch as he takes it dishonestly.
        (l) A takes an article belonging to Z out of Z's possession, without Z's consent, with the intention of keeping it until he obtains money from Z as a reward for its restoration. Here A takes dishonestly; A has therefore committed theft. 
        (m) A, being on friendly terms with Z, goes into Z's library in Z's absence, and takes away a book without Z's express consent for the purpose merely of reading it, and with the intention of returning it. Here, it is probable that A may have conceived that he had Z's implied consent to use Z's book. If this was A's impression, A has not committed theft.
        (n) A, asks charity from Z's wife. She gives A money, food and clothes, which A knows to belong to Z her husband. Here it is probable that A may conceive that Z's wife is authorised to give away alms. If this was A's impression, A has not committed theft. 
        (o) A is the paramour of Z's wife. She gives a valuable property, which A knows to belong to her husband Z, and to be such property as she has no authority from Z to give. If A takes the property dishonestly, he commits theft.
        (p) A, in good faith, believing property belonging to Z to be A's own property, takes that property out of B's possession. Here, as A does not take dishonestly, he does not commit theft.

        """,
        metadata={
            "law":"IPC section 378. Theft",
            "content":"illustrations for the IPC act 378 - theft",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
         Whoever commits theft shall be punished with imprisonment of either description for a term which may extend to three years, or with fine, or with both
        """,
        metadata={
            "law":"IPC section 379. Punishment for theft",
            "content":"law definition for IPC section 379. Punishment for theft.",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        STATE AMENDMENTS of IPC section 379 in Haryana.
        Haryana.—In its application to the State of Haryana, after Section 379, the following sections shall be inserted, namely:—
        “379-A. Snatching.—(1) Whoever, with the intention to commit theft, suddenly or quickly or forcibly seizes or secures or grabs or takes away from any person or from his possession any moveable property, and makes or attempts to make escape with such property, is said to commit snatching. Whoever, commits snatching, shall be punished with rigorous imprisonment for a term, which shall not be less than five years but which may extend to ten years, and shall also be liable to fine of Rupees Twenty five thousand. 
        379-B. Snatching with hurt, wrongful restraint or fear of hurt. Whoever, in order to commit snatching, or in committing the snatching, causes hurt or wrongful restraint or fear of hurt; or after committing the offence of snatching, causes hurt or wrongful restraint or fear of hurt in order to effect his escape, shall be punished with rigorous imprisonment which shall not be less than ten years but which may extend to fourteen years, and shall also be liable to fine of Rupees Twenty five thousand.” [Vide Penal Code (Haryana Amendment) Act, 2014 (Haryana Act No. 18 of 2015), S. 2.]

        """,
        metadata={
            "law":"IPC section 379. Punishment for theft",
            "content":"state amendments of IPC section 379. Punishment for theft in Haryana",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        STATE AMENDMENTS of IPC section 379 in Gujarat.
        —In its application to the State of Gujarat, after Section 379, the following sections shall be inserted, namely:— “379-A. Snatching.—
        (1) Whoever, with the intention to commit theft, suddenly or quickly or forcibly seizes or secures or grabs or takes away from any person or from his physical possession any moveable property, and makes or attempt to make escape with such property, is said to commit snatching. 
        (2) Whoever attempts to commit snatching shall be punished with rigorous imprisonment for a term which shall not be less than five years but which may extend to ten years, and with fine which may extend to twenty five thousand rupees. 
        (3) Whoever commits snatching shall be punished with rigorous imprisonment for a term which shall not be less than seven years but which may extend to ten years, and with fine which may extend to twenty five thousand rupees. 
        (4) Whoever, after committing or attempting to commit snatching, causes hurt or wrongful restraint or fear of hurt, in order to effect his escape shall be punished with rigorous imprisonment for a term which may extend to three years, in addition to the punishment provided for the offence of snatching by the preceding sub-sections. 379-B. Snatching after preparation made for causing death, hurt or restraint in order to the committing of the snatching.— Whoever commits or attempts to commit snatching, having made preparation for causing death, or hurt, or restraint, or fear of death, or of hurt, or of restraint, to any person, in order to the committing of such snatching, or in order to the retaining of property taken by such snatching, shall be punished with rigorous imprisonment for a term which shall not be less than seven years but which may extend to ten years, and with fine which may extend to twenty five thousand rupees.”. [Vide Guj. Act 6 of 2019, S. 2, assented on 21-5- 2019]

        """,
        metadata={
            "law":"IPC section 379. Punishment for theft",
            "content":"state amendments of IPC section 379. Punishment for theft in Gujarat",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        STATE AMENDMENTS of IPC section 379 in Punjab.
        Punjab.—In its application to the State of Punjab, after Section 379, the following sections shall be inserted, namely— “379-A. Snatching.—Whoever, with the intention to commit theft, suddenly or quickly or forcibly seizes or secures or grabs or takes away from any person or from his possession any moveable property, and makes or attempts to make escape with such property, is said to commit snatching. 379-B. Punishment for snatching.—(1) Whoever, commits snatching, shall be punished with imprisonment for a term, which shall not be less than five years, but which may extend to ten years and shall also be liable to fine of rupees ten thousand. (2) If in order to the committing of snatching, or in committing the snatching, the offender causes hurt, or wrongful restraint or fear of hurt or after committing the offence of snatching, causes hurt or wrongful restraint or fear of hurt in order to effect his escape, shall be punished with imprisonment for a term, which shall not be less than ten years and shall also be liable to fine of rupees ten thousand.”. [Vide: Punjab Act 31 of 2014, S. 2, dt. 5-11-2014]. 
        """,
        metadata={
            "law":"IPC section 379. Punishment for theft",
            "content":"state amendments of IPC section 379. Punishment for theft in Punjab",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        AMENDMENTS of IPC section 379 in Chandigrah.
        Union Territory of Chandigarh.—In its application to the Union Territory of Chandigarh, after Section 379, the following sections shall be inserted, namely:— “379-A. Snatching.—(1) Whoever, with the intention to commit theft, suddenly or quickly or forcibly seizes or secures or grabs or takes away from any person or from his possession any moveable property, and makes or attempts to make escape with such property, is said to commit snatching. Whoever, commits snatching, shall be punished with rigorous imprisonment for a term, which shall not be less than five years but which may extend to ten years, and shall also be liable to fine of Rupees Twenty five thousand. 379-B. Snatching with hurt, wrongful restraint or fear of hurt. Whoever, in order to commit snatching, or in committing the snatching, causes hurt or wrongful restraint or fear of hurt; or after committing the offence of snatching, causes hurt or wrongful restraint or fear of hurt in order to effect his escape, shall be punished with rigorous imprisonment which shall not be less than ten years but which may extend to fourteen years, and shall also be liable to fine of Rupees Twenty five thousand.” [Vide G.S.R. 383(E), dated 29-5-2019 (w.e.f. 29-5-2019).]
        """,
        metadata={
            "law":"IPC section 379. Punishment for theft",
            "content":"state amendments of IPC section 379. Punishment for theft in Chandigrah",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        IPC Section 380. Theft in dwelling house, etc.—Whoever commits theft in any building, tent or vessel, which building, tent or vessel is used as a human dwelling, or used for the custody of property, shall be punished with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine.
        """,
        metadata={
            "law":"IPC Section 380. Theft in dwelling house, etc",
            "content":"law definition for IPC Section 380. Theft in dwelling house, etc.",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        STATE AMENDMENTS for IPC section 380 Tamil Nadu.—In its application to the State of Tamil Nadu, Section 380 shall be renumbered as sub-section (1) of that section and after sub-section (1) as so renumbered, the following sub-section shall be added, namely:— “(2) Whoever commits theft in respect of any idol or icon in any building used as a place of worship shall be punished with rigorous imprisonment for a term which shall not be less than two years but which may extend to three years and with fine which shall not be less than two thousand rupees: Provided that the court may, for adequate and special reasons to be mentioned in the judgment, impose a sentence of imprisonment for a term of less than two years.” [Vide Tamil Nadu Act 28 of 1993, S. 2 (w.e.f. 13-7-1993).]
        """,
        metadata={
            "law":"IPC Section 380. Theft in dwelling house, etc",
            "content":"state amendments of IPC Section 380. Theft in dwelling house, etc in Tamil Nadu",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        IPC section 381. Theft by clerk or servant of property in possession of master.— Whoever, being a clerk or servant, or being employed in the capacity of a clerk or servant, commits theft in respect of any property in the possession of his master or employer, shall be punished with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine. 
        """,
        metadata={
            "law":"IPC section 381. Theft by clerk or servant of property in possession of master",
            "content":"law definition for IPC section 381. Theft by clerk or servant of property in possession of master",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        IPC section 382. Theft after preparation made for causing death, hurt or restraint in order to the committing of the theft.—Whoever commits theft, having made preparation for causing death, or hurt, or restraint, or fear of death, or of hurt, or of restraint, to any person, in order to the committing of such theft, or in order to the effecting of his escape after the committing of such theft, or in order to the retaining of property taken by such theft, shall be punished with rigorous imprisonment for a term which may extend to ten years, and shall also be liable to fine.  
        """,
        metadata={
            "law":"IPC section 382. Theft after preparation made for causing death, hurt or restraint in order to the committing of the theft",
            "content":"law definition for IPC section 382. Theft after preparation made for causing death, hurt or restraint in order to the committing of the theft",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        Illustrations  for IPC section 382
        A commits theft on property in Z's possession; and, while committing this theft, he has a loaded pistol under his garment, having provided this pistol for the purpose of hurting Z in case Z should resist.
        A has committed the offence defined in this section. A picks Z's pocket, having posted several of his companions near him, in order that they may restrain Z, if Z should perceive what is passing and should resist, or should attempt to apprehend A. A has committed the offence defined in this section.  
        """,
        metadata={
            "law":"IPC section 382. Theft after preparation made for causing death, hurt or restraint in order to the committing of the theft",
            "content":"illustrations for IPC section 382. Theft after preparation made for causing death, hurt or restraint in order to the committing of the theft",
            "category":"offences against propery of theft"
        }
    ),

    Document(
        page_content="""
        IPC section 383. Extortion.—Whoever intentionally puts any person in fear of any injury to that person, or to any other, and thereby dishonestly induces the person so put in fear to deliver to any person any property or valuable security or anything signed or sealed which may be converted into a valuable security, commits “extortion”.  
        """,
        metadata={
            "law":"IPC section 383- Extortion",
            "content":"law definition for IPC section 383 - Extortion",
            "category":"extortion"
        }
    ),

    Document(
        page_content="""
        Illustrations for IPC section 383 - extortion
        (a) A threatens to publish a defamatory libel concerning Z unless Z gives him money. He thus induces Z to give him money. A has committed extortion. 
        (b) A threatens Z that he will keep Z's child in wrongful confinement, unless Z will sign and deliver to A a promissory note binding Z to pay certain money to A. Z signs and delivers the note. A has committed extortion. 
        (c) A threatens to send club-men to plough up Z's field unless Z will sign and deliver to B a bond binding Z under a penalty to deliver certain produce to B, and thereby induces Z to sign and deliver the bond. A has committed extortion. 
        (d) A, by putting Z in fear of grievous hurt, dishonestly induces Z to sign or affix his seal to a blank paper and deliver it to A. Z signs and delivers the paper to A. Here, as the paper so signed may be converted into a valuable security, A has committed extortion. 

        """,
        metadata={
            "law":"IPC section 383- Extortion",
            "content":"illustrations for IPC section 383 - Extortion",
            "category":"extortion"
        }
    ),

    Document(
        page_content="""
        IPC section 384. Punishment for extortion—Whoever commits extortion shall be punished with imprisonment of either description for a term which may extend to three years, or with fine, or with both. 

        """,
        metadata={
            "law":"IPC section 384. Punishment for extortion",
            "content":"law definition of IPC section 384. Punishment for extortion",
            "category":"extortion"
        }
    ),

    Document(
        page_content="""
        IPC section 385. Putting person in fear of injury in order to commit extortion.— Whoever, in order to the committing of extortion, puts any person in fear, or attempts to put any person in fear, of any injury, shall be punished with imprisonment of either description for a term which may extend to two years, or with fine, or with both.
        """,
        metadata={
            "law":"IPC section 385. Putting person in fear of injury in order to commit extortion",
            "content":"law definition of IPC section 385. Putting person in fear of injury in order to commit extortion",
            "category":"extortion"
        }
    ),

    Document(
        page_content="""
        IPC section 386. Extortion by putting a person in fear of death or grievous hurt. —Whoever commits extortion by putting any person in fear of death or of grievous hurt to that person or to any other, shall be punished with imprisonment of either description for a term which may extend to ten years, and shall also be liable to fine. 
        """,
        metadata={
            "law":"IPC section 386. Extortion by putting a person in fear of death or grievous hurt",
            "content":"law definition of IPC section 386. Extortion by putting a person in fear of death or grievous hurt",
            "category":"extortion"
        }
    ),

    Document(
        page_content="""
        IPC section 387. Putting person in fear of death or of grievous hurt, in order to commit extortion.—Whoever, in order to the committing of extortion, puts or attempts to put any person in fear of death or of grievous hurt to that person or to any other, shall be punished with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine .
        """,
        metadata={
            "law":"IPC section 387. Putting person in fear of death or of grievous hurt, in order to commit extortion",
            "content":"law definition of IPC section 387. Putting person in fear of death or of grievous hurt, in order to commit extortion",
            "category":"extortion"
        }
    ),

    Document(
        page_content="""
        IPC section 388. Extortion by threat of accusation of an offence punishable with death or imprisonment for life, etc.—Whoever commits extortion by putting any person in fear of an accusation against that person or any other, of having committed or attempted to commit any offence punishable with death, or with 398[imprisonment for life], or with imprisonment for a term which may extend to ten years, or of having attempted to induce any other person to commit such offence, shall be punished with imprisonment of either description for a term which may extend to ten years, and shall also be liable to fine; and, if the offence be one punishable under Section 377 of this Code, may be punished with 399[imprisonment for life]. 
        """,
        metadata={
            "law":"IPC section 388. Extortion by threat of accusation of an offence punishable with death or imprisonment for life, etc",
            "content":"law definition of IPC section 388. Extortion by threat of accusation of an offence punishable with death or imprisonment for life, etc"
        }
    ),

    Document(
        page_content="""
        IPC section 389. Putting person in fear of accusation of offence, in order to commit extortion.—Whoever, in order to the committing of extortion, puts or attempts to put any person in fear of an accusation, against that person or any other, of having committed, or attempted to commit, an offence punishable with death or with 400[imprisonment for life], or with imprisonment for a term which may extend to ten years, shall be punished with imprisonment of either description for a term which may extend to ten years, and shall also be liable to fine; and, if the offence be punishable under Section 377 of this Code, may be punished with 401[imprisonment for life].
        """,
        metadata={
            "law":"IPC section 389. Putting person in fear of accusation of offence, in order to commit extortion",
            "content":"IPC section 389. Putting person in fear of accusation of offence, in order to commit extortion"
        }
    ),
]

In [4]:
!pip install -q --upgrade langchain
!pip install -q --upgrade langchain_community


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-objectbox 0.1.0 requires langchain-core<0.2.0,>=0.1.45, but you have langchain-core 0.2.22 which is incompatible.


In [5]:
! pip install lark
! pip install lark-parser
import lark
print(lark.__version__)

0.12.0


In [6]:
from langchain_chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
import lark

metadata_field_info = [
    AttributeInfo(
        name="law",
        description="the Indian Penal Code section that is being discussed in the document",
        type="string",
    ),
    AttributeInfo(
        name="content",
        description="The type of content in the document- [law definition , exception, explanation , state amendments , key points]",
        type="integer",
    ),
    AttributeInfo(
        name="category",
        description="The category of the described crime or law",
        type="string",
    ),
]

document_content_description="List of Indian laws applicable on sexual offences , theft and extortion"
llm=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it",temperature=0)
vectorstore=Chroma.from_documents(sexual_offences_docs,embeddings)
vectorstore=Chroma.from_documents(theft,embeddings)
sqretriever=SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=False, # why maximum 4 
)


NameError: name 'groq_api_key' is not defined

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
def answer(input):
    llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="gemma-7b-it",temperature=0)

    prompt = ChatPromptTemplate.from_template("""
    ##ROLE##
    You are an experienced lawyer,
    specializing in providing legal guidance on various law-related queries. 
    Your task is to understand the client's current case scenario and offer advice based on the given content.

    ##INSTRUCTIONS##
    1) Answer the following question based on the provided context.
    2) Think step-by-step before providing a detailed answer.
    3) Ensure the answer is thorough and helpful.
    4) the provided context is fictional , you have to provide legal advice on the provided fictional case-
    <context>
    {context}
    </context>
    based on the case provide legal help by listing the applicable laws and their definitions
    ficntional case Question: ({input})
    be specific with the laws , 
    provide the most accurate law that is applicable in India
    """)

    document_chain=create_stuff_documents_chain(
        llm,
        prompt,
    )
    retrieval_chain=create_retrieval_chain(
        sqretriever,
        document_chain,
    )
    response=retrieval_chain.invoke({"input":input})
    return response['answer']

In [19]:
description="""

"""
pprint(sqretriever.invoke(description))
print(answer(description))


[Document(metadata={'category': 'offences against propery of theft', 'content': 'state amendments of IPC section 379. Punishment for theft in Haryana', 'law': 'IPC section 379. Punishment for theft'}, page_content='\n        STATE AMENDMENTS of IPC section 379 in Haryana.\n        Haryana.—In its application to the State of Haryana, after Section 379, the following sections shall be inserted, namely:—\n        “379-A. Snatching.—(1) Whoever, with the intention to commit theft, suddenly or quickly or forcibly seizes or secures or grabs or takes away from any person or from his possession any moveable property, and makes or attempts to make escape with such property, is said to commit snatching. Whoever, commits snatching, shall be punished with rigorous imprisonment for a term, which shall not be less than five years but which may extend to ten years, and shall also be liable to fine of Rupees Twenty five thousand. \n        379-B. Snatching with hurt, wrongful restraint or fear of hurt